In [ ]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import nltk
from sklearn import cluster
from sklearn import metrics
from sklearn.cluster import SpectralClustering
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
root = os.getcwd()+'/'
paperFolder = 'papers/'
stopw = stopwords.words('english')

In [ ]:
def clean(path, text0, stopwords):
    ##returns a list of lists, with words in it.
    with open(path+text0) as txt:
        data = txt.read()

        # data = re.sub('\\w+\\set\\sal', '<UNK>', data)
        data = re.sub('\\w+\\sand\\s+\\w+\s\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\s+\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('\\([^)]*\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\\set\\sal.', '<UNK>', data)
        data = re.sub('\s+[.]', '.', data)
        data = re.sub('\s+[,]', ',', data)
        data = data.replace(u'\xa0', u' ')
        data = data.split('\n')
        data = [n.strip().split('.') for n in data]
        data = [[z.strip().split(' ') for z in n] for n in data]
        data = [[z for z in n if len(z) > 5] for n in data if len(n) > 0]
        data = [n for n in data if len(n) > 0]
        data = [[n for sublist in l for n in sublist] for l in data]
        data = [[n.lower() for n in l if not re.search('\\d', n)] for l in data]
        data = [[n for n in l if re.search('[a-zA-Z]', n)] for l in data] ####TODO check
        data = [[n for n in l if n not in stopwords] for l in data]
        data = [[n for n in l if '<unk>' not in n] for l in data]
        data = [[n for n in l if len(n)>1] for l in data]
        data = [[n.replace(',','') for n in l] for l in data]
        data = [[n.replace("'s",'') for n in l] for l in data]
        # data = [[n.split('-') for n in l] for l in data]
        # data = [[n for n in l if not re.search('\\', n)] for l in data]
        txt.close()
    return data


def labeledSentences(tokenizedDescriptions, labels = None):
    sentences = [gensim.models.doc2vec.TaggedDocument(words = i, tags = [labels[n]]) for n, i in enumerate(tokenizedDescriptions)]
    return sentences

In [ ]:
#### Developing Model
trainList = np.random.choice(os.listdir(root+paperFolder), 1000, replace = False).tolist()
trainPapers = [clean(root+paperFolder, n, stopw) for n in trainList]

##labels
labels = list(range(np.sum([len(n) for n in trainPapers])))
labels = ['lab_{}'.format(n) for n in labels]
len(labels)

##sentences
sentences = []
for n in trainPapers:
    for m in n:
        sentences.append(m)
len(sentences)


## Doc2Vec Model
labSen = labeledSentences(sentences, labels = labels)

model = gensim.models.Doc2Vec(size = 100,
                              window = 3,
                              sample = 1e-3,
                              min_count = 1,
                              workers = 8,
                              seed = 1,
                              dm_concat = 1,
                              dm_mean = 1,
                              dm = 1)
model.build_vocab(labSen)